In [23]:
import torch
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()


Using cache found in /home/nclhpm/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [24]:
#torch.cuda.get_device_name(0)
#print("\n")
torch.cuda.is_available()

False

In [25]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [26]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([ 2.0977e+00, -1.7348e+00, -2.2355e+00, -2.9669e+00, -2.3805e+00,
         9.7397e-01, -1.6049e+00,  3.6914e+00,  6.3812e+00, -1.2929e+00,
        -6.7555e+00, -3.3525e+00, -7.9619e+00, -4.4554e+00, -5.6423e+00,
        -4.6624e+00, -1.9577e+00, -3.5811e-01, -1.2812e+00, -4.6707e+00,
        -3.2935e+00, -2.5674e+00, -2.4351e+00, -1.3017e+00, -3.2453e+00,
        -1.4237e+00, -1.2001e+00,  4.1275e-01, -1.6093e+00,  1.5871e+00,
         2.7725e-01, -6.2652e-01, -2.9734e-01, -3.8219e+00, -1.5450e+00,
        -2.8976e+00, -5.6528e-01, -2.3938e+00, -3.3705e-01,  1.2809e+00,
        -1.2516e+00, -2.6469e+00, -3.1011e+00, -2.2447e+00, -4.4385e-01,
        -1.2620e+00,  8.2895e-01, -2.0436e+00, -6.6037e-01, -8.6522e-02,
         4.8967e-01, -1.7190e+00, -7.7943e-01, -1.1046e+00, -5.3857e-01,
        -2.9254e+00, -1.9327e+00, -2.7273e+00, -6.0903e-01, -1.6802e+00,
         1.3443e+00, -4.2062e+00, -1.4768e+00, -4.5581e+00, -3.2726e+00,
        -4.0086e+00,  1.5702e-01, -1.9921e+00, -7.4

In [28]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2022-12-12 18:56:41--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.3’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.003s  

2022-12-12 18:56:41 (2.94 MB/s) - ‘imagenet_classes.txt.3’ saved [10472/10472]



In [27]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 1000)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())


Samoyed 0.8303046226501465
Pomeranian 0.06988755613565445
keeshond 0.01296406053006649
collie 0.010797739028930664
Great Pyrenees 0.009886739775538445
Shetland sheepdog 0.008079971186816692
Eskimo dog 0.007716516498476267
Border collie 0.006666215136647224
Siberian husky 0.005079002119600773
Arctic fox 0.004981102887541056
West Highland white terrier 0.004850371275097132
schipperke 0.003958225250244141
white wolf 0.003950091544538736
malamute 0.00391095457598567
Angora 0.0024832526687532663
wallaby 0.002022006781771779
Pembroke 0.0019301521824672818
papillon 0.0014838646166026592
groenendael 0.0014537437818944454
kuvasz 0.0006565293879248202
Persian cat 0.000547415460459888
Old English sheepdog 0.0003379475383553654
Scotch terrier 0.0003294127818662673
hen 0.0002858813968487084
Japanese spaniel 0.0002712786372285336
hare 0.0002564542228356004
Chihuahua 0.00022618153889197856
Cardigan 0.00019642522966023535
chow 0.00019524042727425694
Norwegian elkhound 0.00016818262520246208
croquet ba

In [34]:
from torchvision import datasets
from torchvision.io import read_image
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights 
from torchvision import transforms
import numpy as np
import time
import os
import matplotlib.pyplot as plt
path = '/home/nclhpm/Desktop/Teymoor/CNNBenchmark/RESNET50/Dataset/'
img_names = os.listdir(path)

import subprocess

#command = 'sudo ./test.sh -S Fri16Sep'
#process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
#output, error = process.communicate()
#torch.cuda.is_available = lambda : False
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sudo_password = 'Fri16Sep'
command = 'sudo ./test.sh'
command = command.split()

cmd1 = subprocess.Popen(['echo',sudo_password], stdout=subprocess.PIPE)
cmd2 = subprocess.Popen(['sudo','-S'] + command, stdin=cmd1.stdout, stdout=subprocess.PIPE)

output = cmd2.stdout.read().decode() 
print(output)
torch.cuda.is_available()

for idx, name in enumerate(img_names):
    start = time.time()
    img_name = path + name
    # Use you favourite library to load the image
    img = read_image(img_name)
    weights = MobileNet_V2_Weights.DEFAULT
    model = mobilenet_v2(weights=weights)
    model.eval()

# Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
    batch = preprocess(img).unsqueeze(0).cpu()
   
    if torch.cuda.is_available():
      batch = batch.to('cuda')
      model.to('cuda')

    with torch.no_grad():
      output = model(batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
      #print(output[0])
   
    # Step 4: Use the model and print the predicted category
    end = time.time()
    prediction = model(batch).squeeze(0).softmax(0).cpu()
    class_id = prediction.argmax().item()
    score = prediction[class_id].item()
    category_name = weights.meta["categories"][class_id]
    print(f"{category_name}: {100 * score:.1f}%")
    print(img_name)
    print(end - start)



sudo_password = 'Fri16Sep'
command = 'sudo ./test.sh'
command = command.split()

cmd1 = subprocess.Popen(['echo',sudo_password], stdout=subprocess.PIPE)
cmd2 = subprocess.Popen(['sudo','-S'] + command, stdin=cmd1.stdout, stdout=subprocess.PIPE)

output = cmd2.stdout.read().decode() 
print(output)
        #image = plt.imread(img_name)
        #images[idx] = image

    #img = read_image("dog.jpg")

    # Step 1: Initialize model with the best available weights



118557090719



AttributeError: 'float' object has no attribute 'cpu'

In [2]:
from torchvision import datasets
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
import numpy as np
import time
import os
import matplotlib.pyplot as plt
path = '/home/nclhpm/Desktop/Teymoor/CNNBenchmark/RESNET50/Dataset/'
img_names = os.listdir(path)

import subprocess

#command = 'sudo ./test.sh -S Fri16Sep'
#process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
#output, error = process.communicate()

sudo_password = 'Fri16Sep'
command = 'sudo ./test.sh'
command = command.split()

cmd1 = subprocess.Popen(['echo',sudo_password], stdout=subprocess.PIPE)
cmd2 = subprocess.Popen(['sudo','-S'] + command, stdin=cmd1.stdout, stdout=subprocess.PIPE)

output = cmd2.stdout.read().decode() 
print(output)
torch.cuda.is_available()

for idx, name in enumerate(img_names):
    start = time.time()
    img_name = path + name
    # Use you favourite library to load the image
    img = read_image(img_name)
    weights = ResNet50_Weights.DEFAULT
    model = resnet50(weights=weights)
    model.eval()

# Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
    batch = preprocess(img).unsqueeze(0)
    
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
      #print(output[0])
   
    # Step 4: Use the model and print the predicted category
    end = time.time()
    prediction = model(batch).squeeze(0).softmax(0)
    class_id = prediction.argmax().item()
    score = prediction[class_id].item()
    category_name = weights.meta["categories"][class_id]
    print(f"{category_name}: {100 * score:.1f}%")
    print(img_name)
    print(end - start)



sudo_password = 'Fri16Sep'
command = 'sudo ./test.sh'
command = command.split()

cmd1 = subprocess.Popen(['echo',sudo_password], stdout=subprocess.PIPE)
cmd2 = subprocess.Popen(['sudo','-S'] + command, stdin=cmd1.stdout, stdout=subprocess.PIPE)

output = cmd2.stdout.read().decode() 
print(output)
        #image = plt.imread(img_name)
        #images[idx] = image

    #img = read_image("dog.jpg")

    # Step 1: Initialize model with the best available weights

NameError: name 'torch' is not defined